In [14]:
from Bio.Seq import Seq
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import numpy as np
import re
import os
import gzip

In [64]:
i5_index = {
    "D501": "AGGCTATA",
    "D502": "GCCTCTAT",
    "D503": "AGGATAGG",
    "D504": "TCAGAGCC",
    "D505": "CTTCGCCT",
    "D506": "TAAGATTA",
    "D507": "ACGTCCTG",
    "D508": "GTCAGTAC"
}
i7_index = {
    "D701": "ATTACTCG",
    "D702": "TCCGGAGA", 
    "D703": "CGCTCATT", 
    "D704": "GAGATTCC", 
    "D705": "ATTCAGAA",
    "D706": "GAATTCGT",
    "D707": "CTGAAGCT",
    "D708": "TAATGCGC",
    "D709": "CGGCTATG",
    "D710": "TCCGCGAA",
    "D711": "TCTCGCGC",
    "D712": "AGCGATAG"
}

In [90]:
total_line = 11173943428

In [139]:
dual_indices = [i7_index["D701"] + "_" +i5_index["D501"],
               i7_index["D702"] + "_" +i5_index["D502"],
               i7_index["D703"] + "_" +i5_index["D503"],
               i7_index["D704"] + "_" +i5_index["D504"],
               i7_index["D705"] + "_" +i5_index["D505"], 
               i7_index["D706"] + "_" +i5_index["D506"],
               i7_index["D707"] + "_" +i5_index["D507"],
               i7_index["D708"] + "_" +i5_index["D508"],
               i7_index["D709"] + "_" +i5_index["D502"],
               i7_index["D710"] + "_" +i5_index["D506"],
               i7_index["D708"] + "_" +i5_index["D505"],
               i7_index["D707"] + "_" +i5_index["D504"],
               i7_index["D706"] + "_" +i5_index["D503"],
               i7_index["D705"] + "_" +i5_index["D502"],
               i7_index["D702"] + "_" +i5_index["D506"],
               i7_index["D701"] + "_" +i5_index["D507"],
               i7_index["D704"] + "_" +i5_index["D508"],
               i7_index["D703"] + "_" +i5_index["D501"],
               i7_index["D712"] + "_" +i5_index["D503"],
               i7_index["D711"] + "_" +i5_index["D507"]
               ]

for i in dual_indices:
    vars()[i] = open(i + ".txt", "w")
f_file = FastqGeneralIterator(gzip.open("./CRC_2_CKDL200166600-1a_HLLTTDSXY_L4_1.fq.gz", "rt"))
r_file = FastqGeneralIterator(gzip.open("./CRC_2_CKDL200166600-1a_HLLTTDSXY_L4_2.fq.gz", "rt"))
seqtag_pos = 0
f_multitag_pos = 8
f_barcode_pos = 57
r_multitag_pos = 7
r_barcode_pos = 32
f_barcode_length = 34
r_barcode_length = 27

recipient_re = re.compile('\D*?(.ACA|G.CA|GA.A|GAC.)\D{4,7}?AA\D{4,7}?AA\D{4,7}?TT\D{4,7}?(.TCG|C.CG|CT.G|CTC.)\D*')
donor_re = re.compile('\D*?(.GGC|T.GC|TG.C|TGG.)\D{4,7}?AA\D{4,7}?TT\D{4,7}?(.CGG|G.GG|GC.G|GCG.)\D*')
recipient_f_clipper = re.compile('(.ACA|G.CA|GA.A|GAC.)')
donor_f_clipper = re.compile('(.GGC|T.GC|TG.C|TGG.)')
recipient_r_clipper = re.compile('(.CTC|G.TC|GC.C|GCT.)')
donor_r_clipper = re.compile('(.GCG|G.CG|GG.G|GGC.)')

min_qs = 25
quality_bps_count = 0

for f_record, r_record in zip(f_file, r_file):
    
    fr = f_record[1]
    rr = r_record[1]
    fq = [ord(i) -33 for i in list(f_record[2])]
    rq = [ord(i) - 33 for i in list(r_record[2])]
    recipient_tag_grep = recipient_re.match(str(fr)[f_barcode_pos : f_barcode_pos + f_barcode_length])
    donor_tag_grep = donor_re.match(str(rr)[r_barcode_pos : r_barcode_pos + r_barcode_length])
    
    if recipient_tag_grep is not None and donor_tag_grep is not None:
        if np.mean(fq[recipient_tag_grep.start() : recipient_tag_grep.end()]) >= min_qs and \
        np.mean(rq[donor_tag_grep.start() : donor_tag_grep.end()]) >= min_qs:
            index_read = f_record[0][-17:].replace("+", "_")
            f_multitag = fr[f_multitag_pos : f_multitag_pos + 6]
            f_seqtag = fr[0:8]
            r_seqtag = rr[0:7]
            r_multitag = rr[r_multitag_pos : r_multitag_pos + 9]
            quality_bps_count += 1
            if index_read in dual_indices:
                vars()[index_read].write(recipient_tag_grep.group()[4:-4] + "," + donor_tag_grep.group()[4:-4] 
                                     + "," + f_multitag + "," + r_multitag + "," + f_seqtag + r_seqtag + "\n")
#             vars()[index_read].write(rr + "\n")


for i in dual_indices:
    vars()[i].close()

In [140]:

print(quality_bps_count)

3790468


In [158]:
f = open("/Users/xianan/Dropbox/BPS_lib/pSL439_donor_bc.txt")
pSL439_donor = []
for line in f.readlines():
    pSL439_donor.append(line.split(",")[0])
f.close()
f = open("/Users/xianan/Dropbox/BPS_lib/pSL438_donor_bc.txt")
pSL438_donor = []
for line in f.readlines():
    pSL438_donor.append(line.split(",")[0])
f.close()

In [191]:
f = open("test_extracted_barcode_cluster.csv")
next(f)
condensed439_donor = []
for line in f.readlines():
    condensed439_donor.append(line.split(",")[1])
f.close()
f = open("ATTACTCG_AGGCTATA_barcodes_pcr_cluster.csv")
next(f)
condensed438_donor_ATTACTCG_AGGCTATA = []
for line in f.readlines():
    condensed438_donor_ATTACTCG_AGGCTATA.append(line.split(",")[1])
f.close()
f = open("CGCTCATT_AGGATAGG_barcodes_pcr_cluster.csv")
next(f)
condensed438_donor_CGCTCATT_AGGATAGG = []
for line in f.readlines():
    condensed438_donor_CGCTCATT_AGGATAGG.append(line.split(",")[1])
f.close()

In [164]:
condensed_439_rc = []
for i in condensed439_donor:
    condensed_439_rc.append(str(Seq(i).reverse_complement()))

In [224]:
f = open("/Users/xianan/Dropbox/BPS_lib/pSL439_donor_bc.txt")
pSL439_BC = []
for i in f.readlines():
    pSL439_BC.append(i.rstrip("\n").split(",")[1])
f.close()
f = open("/Users/xianan/Dropbox/BPS_lib/pSL438_donor_bc.txt")
pSL438_BC = []
for i in f.readlines():
    pSL438_BC.append(i.rstrip("\n").split(",")[1])
f.close()

In [211]:
f = open("ATTACTCG_AGGCTATA_barcodes_pcr_cluster.csv")
next(f)
condensed438_donor_ATTACTCG_AGGCTATA = {}
for line in f.readlines():
    condensed438_donor_ATTACTCG_AGGCTATA[line.rstrip("\n").split(",")[1]] = int(line.rstrip("\n").split(",")[3])
f.close()
f = open("test_extracted_barcode_cluster.csv")
next(f)
condensed439_donor = {}
for line in f.readlines():
    condensed439_donor[line.rstrip("\n").split(",")[1]] = int(line.rstrip("\n").split(",")[3])
f.close()

In [223]:
for k, v in condensed439_donor.items():
    if v >= 1000 and k not in pSL439_BC:
        print(k, v)

ACTTTAAATCGTTTGATTA 1279
GAACAAAGGTCCTTTCCGC 1582
TAAAAAAACTAATTCCAGC 1674


In [227]:
for k, v in condensed438_donor_ATTACTCG_AGGCTATA.items():
    if v >= 700 and k not in pSL438_BC:
        print(k, v)

CCTCTAAATAAGTTGAAAA 772
TATAAAACTTTCTTTAAAC 845
TGTGAAACTAATTTATCAG 1426


In [226]:
for i in pSL438_BC:
    if i not in condensed438_donor_ATTACTCG_AGGCTATA.keys():
        print (i)

CAGGTAACCCATATGCATG
CAGTAAAACGAGTTCTAAA
AGTTGAAGCTTTTTGGTGA
GTCCGAAGGCAGTTCTTCC
GGTCCAAGGAGGTTAGAGA
